In [4]:
import random
import numpy as np

# Simple GRASP implementation for TSP problem
Hyperparms : alpha (affects directly the RCL), iterations number, seed (for reproductible results)

In [5]:
class TSP:
    def __init__(self, cities, alpha=0.2, iterations=100, seed=None):
        """
        Initializes the TSP problem instance.
        
        Parameters:
        cities (list of tuples): Each city is represented by a tuple (index, x, y).
        alpha (float): A factor that controls the size of the Restricted Candidate List (RCL).
        iterations (int): The number of iterations for the GRASP algorithm.
        seed (int, optional): Seed for random number generation to ensure reproducibility.
        
        Sets the random seed for consistent results (if seed is provided).
        """
        self.cities = cities
        self.alpha = alpha
        self.iterations = iterations
        self.seed = seed
        self.best_tour = None
        self.best_distance = float('inf')

        # Set random seed for reproducibility (if provided)
        if self.seed is not None:
            random.seed(self.seed)
            np.random.seed(self.seed)

    def euclidean_distance(self, city1, city2):
        """
        Calculates the Euclidean distance between two cities.
        
        Parameters:
        city1, city2 (tuple): Each city is represented by a tuple (index, x, y).
        
        Returns:
        float: The Euclidean distance between the two cities.
        """
        return np.sqrt((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)

    def total_distance(self, tour):
        """
        Calculates the total distance of a tour, including returning to the starting city.
        
        Parameters:
        tour (list): List of cities visited in order.
        
        Returns:
        float: The total distance of the tour.
        """
        distance = 0
        # Sum the distances between consecutive cities
        for i in range(len(tour) - 1):
            distance += self.euclidean_distance(tour[i], tour[i+1])
        # Add the distance from the last city back to the start city
        distance += self.euclidean_distance(tour[-1], tour[0])  
        return distance

    def build_rcl(self, tour):
        """
        Constructs the Restricted Candidate List (RCL) based on a greedy approach.
        
        Parameters:
        tour (list): The current partial tour (cities visited so far).
        
        Returns:
        list: The RCL, which contains cities based on proximity to the last visited city.
        """
        rcl = []
        last_city = tour[-1]  # Last city visited in the current tour
        distances = []
        
        # Calculate the distance from the last city to all unvisited cities
        for city in self.cities:
            if city not in tour:  # Don't consider cities already in the tour
                dist = self.euclidean_distance(last_city, city)
                distances.append((city, dist))
        
        # Sort the cities by distance from the last city
        distances.sort(key=lambda x: x[1])

        # Select the cutoff distance based on the alpha parameter
        min_distance = distances[0][1]
        max_distance = distances[-1][1]
        rcl_cutoff = min_distance + self.alpha * (max_distance - min_distance)

        # Filter cities based on the RCL cutoff
        rcl = [city for city, dist in distances if dist <= rcl_cutoff]
        
        return rcl

    def local_search(self, tour):
        """
        Attempts to improve the current tour using the 2-opt swap technique.
        
        Parameters:
        tour (list): The current tour (list of cities).
        
        Returns:
        list: The best tour found after trying all 2-opt swaps.
        """
        best_tour = tour
        best_distance = self.total_distance(tour)

        # Try all 2-opt swaps (reversing segments of the tour)
        for i in range(len(tour)):
            for j in range(i + 2, len(tour) - 1):  # Avoid adjacent cities
                if j - i == 1:
                    continue
                new_tour = tour[:i+1] + tour[i+1:j+1][::-1] + tour[j+1:]
                new_distance = self.total_distance(new_tour)
                if new_distance < best_distance:
                    best_tour = new_tour
                    best_distance = new_distance

        return best_tour

    def grasp(self):
        """
        Solves the TSP using the GRASP (Greedy Randomized Adaptive Search Procedure) algorithm.
        
        The algorithm iteratively constructs a tour using the RCL and then improves it using local search.
        
        Returns:
        tuple: The best tour and its total distance found by the algorithm.
        """
        # Run the algorithm for the specified number of iterations
        for iteration in range(self.iterations):
            # Construction Phase: Build a tour
            tour = [self.cities[0]]  # Start with the first city
            while len(tour) < len(self.cities):
                rcl = self.build_rcl(tour)  # Build RCL based on current tour
                next_city = random.choice(rcl)  # Randomly pick a city from RCL
                tour.append(next_city)
            
            # Local Search Phase: Try improving the tour
            improved_tour = self.local_search(tour)
            improved_distance = self.total_distance(improved_tour)
            
            # Update the best solution found
            if improved_distance < self.best_distance:
                self.best_tour = improved_tour
                self.best_distance = improved_distance

        return self.best_tour, self.best_distance

In [6]:
# Example usage:
cities = [
    (0, 0, 0), (1, 1, 5), (2, 2, 2), (3, 3, 7), (4, 4, 3),
    (5, 5, 10), (6, 6, 8), (7, 7, 1), (8, 8, 6), (9, 9, 4)
]

# Initialize TSP problem with a specific seed for reproducibility
seed_value = 42  # This seed ensures the same random choices in each run
tsp_problem = TSP(cities, alpha=0.2, iterations=100, seed=seed_value)

# Solve the TSP using GRASP
best_tour, best_distance = tsp_problem.grasp()

# Output the result (best tour and its total distance)
print(f"Best Tour: {best_tour}")
print(f"Best Distance: {best_distance:.2f}")

Best Tour: [(0, 0, 0), (2, 2, 2), (4, 4, 3), (7, 7, 1), (9, 9, 4), (8, 8, 6), (6, 6, 8), (5, 5, 10), (3, 3, 7), (1, 1, 5)]
Best Distance: 31.11
